# Projeto Final — Fluxo Integrado de Preparação e Modelagem
Este notebook reúne todo o trabalho desenvolvido nos arquivos originais, mas agora organizado em um único fluxo com comentários mais detalhados. 

Ao longo das seções, indicamos o que já havia sido feito, o que foi ajustado nesta revisão e por quê, sempre com uma linguagem pensada para estudantes. Dessa forma, você consegue acompanhar o raciocínio e adaptar o material com facilidade nas próximas etapas do curso.


## Visão Geral do Notebook
1. **Configuração do ambiente e importações** — instalamos/ativamos as bibliotecas necessárias e explicamos para que serve cada grupo de ferramentas.
2. **Coleta dos dados no BigQuery** — mantemos o passo original para buscar os dados oficiais e explicamos como habilitar a autenticação quando estiver usando a nuvem.
3. **Uso temporário de arquivos locais (`data/`)** — incluímos uma alternativa para rodar tudo apenas com os CSVs do repositório, o que facilita os testes e a validação neste momento.
4. **Análise exploratória** — revisamos as tabelas básicas, destacamos o que os gráficos mostram e apontamos os fatores de maior impacto.
5. **Preparação dos dados** — detalhamos como tratar valores ausentes, lidar com outliers sem perder observações e padronizar categorias raras.
6. **Modelagem e validação** — treinamos o Random Forest (mesmo modelo usado antes) e agora adicionamos validação cruzada e um baseline simples para comparar resultados.
7. **Simulações e previsão no conjunto de teste** — reaproveitamos o pipeline para simular perfis de pacientes e prever as internações do conjunto de teste.
8. **Recomendações finais** — reunimos os aprendizados e próximos passos sugeridos.


# 0. Configurações e Importações
Nesta etapa, garantimos que todas as dependências estejam disponíveis. O objetivo é rodar o notebook tanto no Google Colab (com BigQuery) quanto localmente, por isso mantivemos os mesmos pacotes e acrescentamos alguns utilitários para validação dos modelos.


In [ ]:
!pip install google-cloud-bigquery --quiet
!pip install pandas scikit-learn matplotlib seaborn joblib --quiet

from pathlib import Path

from google.cloud import bigquery
from google.colab import auth

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor

import joblib

%matplotlib inline


## 0.1 Autenticação no BigQuery

Autentique-se utilizando as credenciais do Google para acessar o projeto do BigQuery.


In [ ]:
auth.authenticate_user()


# 1. Coleta dos Dados no BigQuery

Consultamos as tabelas de treino e teste diretamente no projeto `t1engenhariadados` para garantir que utilizamos a versão mais atualizada dos dados.


In [ ]:
main_project = bigquery.Client(project="t1engenhariadados")


In [ ]:
    select_train_data = """"
SELECT *
FROM `t1engenhariadados.turma2_grupo9.train_data`
"""
    train = main_project.query(select_train_data).to_dataframe()

    select_test_data = """"
SELECT *
FROM `t1engenhariadados.turma2_grupo9.test_data`
"""
    test = main_project.query(select_test_data).to_dataframe()

    print(f"Dimensões do conjunto de treino: {train.shape}")
    print(f"Dimensões do conjunto de teste:  {test.shape}")


### 1.2 Usando os arquivos locais do repositório (passo opcional)
Durante o desenvolvimento, nem sempre teremos acesso ao BigQuery. Para continuar os estudos mesmo assim, adicionamos esta alternativa usando os CSVs disponíveis na pasta `data/`. 

* **O que mudou?** Nada foi removido do fluxo original: apenas acrescentamos esta etapa opcional.
* **Como funciona?** Definimos uma variável `USE_LOCAL_DATA`. Quando ela estiver como `True`, carregamos os arquivos locais e sobrescrevemos `train` e `test`. Quando estiver como `False`, mantemos exatamente o que veio do BigQuery.
* **Por que isso é útil?** Permite validar o modelo agora e deixar a alteração do BigQuery para quando o grupo estiver pronto para subir os dados atualizados.


In [ ]:
USE_LOCAL_DATA = True  # altere para False quando quiser usar somente os dados vindos do BigQuery

data_source = "BigQuery"
if USE_LOCAL_DATA:
    data_source = "arquivos locais (.csv)"
    data_dir = None
    for candidate in [Path("data"), Path("../data")]:
        if (candidate / "train_data.csv").exists() and (candidate / "test_data.csv").exists():
            data_dir = candidate
            break
    if data_dir is None:
        raise FileNotFoundError("Não encontrei os arquivos train_data.csv e test_data.csv na pasta data/.")

    train = pd.read_csv(data_dir / "train_data.csv")
    test = pd.read_csv(data_dir / "test_data.csv")

    print("Carregando dados a partir dos CSVs locais...")
    print(f"Diretório utilizado: {data_dir.resolve()}")
    print(f"Dimensões do conjunto de treino: {train.shape}")
    print(f"Dimensões do conjunto de teste:  {test.shape}")
else:
    print("Mantendo os dados carregados do BigQuery (nenhum arquivo local foi lido).")

print(f"Fonte atual de dados: {data_source}")


## 1.1 Preparação do DataFrame base

Manteremos uma cópia do conjunto de treino para exploração e pré-processamento, preservando os dados brutos retornados do BigQuery.


In [ ]:
df_raw = train.copy()
df_exploration = df_raw.copy()

print(f'Análise baseada na fonte de dados: {data_source}')
df_raw.head()


# 2. Análise Exploratória dos Dados
Antes de modelar, revisamos a estrutura do dataset para entender volumes, tipos de variáveis e possíveis problemas. Mantivemos as consultas originais e adicionamos interpretações sobre o que observar em cada saída.


In [ ]:
df_exploration.info()


In [ ]:
df_exploration.describe(include="all")


## 2.1 Padrões e Relações

Exploramos correlações entre variáveis numéricas e a distribuição de `Stay` em relação a atributos categóricos relevantes.


In [ ]:
numeric_cols = df_exploration.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(10, 6))
sns.heatmap(df_exploration[numeric_cols].corr(), annot=True, cmap="coolwarm")
plt.title("Correlação entre variáveis numéricas")
plt.show()


In [ ]:
    print("Value counts for Stay:")
    print(df_exploration["Stay"].value_counts().to_markdown())

    print("
Stay vs Hospital_type_code:")
    print(df_exploration.groupby("Hospital_type_code")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Department:")
    print(df_exploration.groupby("Department")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Ward_Type:")
    print(df_exploration.groupby("Ward_Type")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Severity of Illness:")
    print(df_exploration.groupby("Severity of Illness")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Age:")
    print(df_exploration.groupby("Age")["Stay"].value_counts(normalize=True).unstack().to_markdown())


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df_exploration, x="Stay", order=df_exploration["Stay"].value_counts().index)
plt.title("Distribuição da Duração da Estadia (Stay)")
plt.xlabel("Duração da Estadia")
plt.ylabel("Contagem")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 7))
sns.countplot(
    data=df_exploration,
    x="Severity of Illness",
    hue="Stay",
    palette="viridis",
    order=df_exploration["Severity of Illness"].value_counts().index,
)
plt.title("Duração da Estadia por Gravidade da Doença")
plt.xlabel("Gravidade da Doença")
plt.ylabel("Contagem")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 2.2 Fatores que influenciam a duração da internação
Os gráficos e tabelas apontam que variáveis como gravidade da doença, tipo de admissão e faixa etária têm forte associação com o tempo de internação. Nesta revisão incluímos notas explicando como interpretar essas relações para apoiar discussões em sala.

- **Gravidade (`Severity of Illness`)**: pacientes classificados como `Extreme` concentram as maiores durações.
- **Tipo de admissão (`Type of Admission`)**: internações de emergência tendem a ser mais longas do que procedimentos eletivos.
- **Recursos do hospital**: hospitais com menos quartos extras aparecem mais nos casos longos, sugerindo gargalos.
- **Depósito de admissão**: mesmo após aplicar o log, valores muito altos continuam associados a estadias maiores, possivelmente por refletir tratamentos mais complexos.


## 2.3 Interação entre idade e gravidade da doença
Esta seção já existia, mas agora destacamos por que olhar para a combinação das variáveis faz diferença. Ao cruzar idade com gravidade, percebemos padrões que não ficam claros quando analisamos cada atributo separadamente.


### Principais insights da interação
1. **Idade e gravidade extrema:** pacientes mais velhos com gravidade `Extreme` concentram as maiores estadias, reforçando a necessidade de monitoramento contínuo.
2. **Casos moderados em adultos jovens:** mesmo com gravidade `Moderate`, adultos na faixa dos 31-50 anos podem permanecer acima da média, sinalizando que fatores clínicos adicionais merecem investigação.
3. **Impacto das admissões de emergência:** quando combinamos idade avançada, gravidade alta e emergência, observamos as maiores filas — informação útil para priorizar recursos.


# 3. Preparação dos Dados para Machine Learning
Retomamos as etapas do notebook de preparação, mas com alguns ajustes para evitar perdas de informação e manter o processo consistente entre treino e inferência.

**Principais melhorias adicionadas agora:**
- Substituímos o preenchimento de `City_Code_Patient` por `0` pelo uso da moda real da coluna, evitando criar uma cidade fictícia.
- Calculamos os limites de outliers uma única vez e aplicamos `clip` (ao invés de remover linhas), preservando todos os registros.
- Reforçamos a função de conversão de idade para garantir que sempre devolva valores numéricos.
- Mantivemos o agrupamento de categorias raras e o log da coluna `Admission_Deposit`, explicando quando usar cada transformação.


In [ ]:
def age_to_numeric(age_range):
    if isinstance(age_range, (int, float)) and not pd.isna(age_range):
        return float(age_range)

    if isinstance(age_range, str):
        if "-" in age_range:
            low, high = age_range.split("-")
            return (float(low) + float(high)) / 2
        digits = "".join(ch for ch in age_range if ch.isdigit())
        if digits:
            return float(digits)

    return np.nan


In [ ]:
df_prepared = df_raw.copy()

# Tratamento de valores ausentes com estatísticas dos dados
bed_grade_mode = df_prepared['Bed Grade'].mode(dropna=True)[0]
df_prepared['Bed Grade'] = df_prepared['Bed Grade'].fillna(bed_grade_mode)

city_code_mode = df_prepared['City_Code_Patient'].mode(dropna=True)[0]
df_prepared['City_Code_Patient'] = (
    pd.to_numeric(df_prepared['City_Code_Patient'], errors='coerce')
    .fillna(city_code_mode)
    .astype(int)
)

# Conversão da idade para valores numéricos médios
df_prepared['Age'] = df_prepared['Age'].apply(age_to_numeric)

# Remoção de duplicatas exatas
df_prepared = df_prepared.drop_duplicates()

# Tratamento de outliers via IQR com recorte (clip)
numeric_cols = df_prepared.select_dtypes(include=np.number).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['case_id', 'patientid']]

outlier_bounds = {}
for col in numeric_cols:
    Q1 = df_prepared[col].quantile(0.25)
    Q3 = df_prepared[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_bounds[col] = (lower, upper)
    df_prepared[col] = df_prepared[col].clip(lower, upper)

# Transformação logarítmica em Admission_Deposit
if 'Admission_Deposit' in df_prepared.columns:
    df_prepared['Admission_Deposit'] = np.log1p(df_prepared['Admission_Deposit'])

# Remoção de variáveis altamente correlacionadas
numeric_cols_clean = df_prepared.select_dtypes(include=np.number).columns.tolist()
corr_matrix = df_prepared[numeric_cols_clean].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
columns_to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.95)]
df_prepared = df_prepared.drop(columns=columns_to_drop_corr, errors='ignore')

# Agrupamento de categorias raras
frequent_categories = {}
cat_cols = df_prepared.select_dtypes(include='object').columns.tolist()
if 'Stay' in cat_cols:
    cat_cols.remove('Stay')

for col in cat_cols:
    freq = df_prepared[col].value_counts(normalize=True)
    raras = freq[freq < 0.01].index
    allowed = set(freq[freq >= 0.01].index)
    if len(raras) > 0:
        df_prepared[col] = df_prepared[col].replace(list(raras), 'OUTRA')
        allowed.add('OUTRA')
    frequent_categories[col] = allowed


In [ ]:
print(f'Dimensões após limpeza: {df_prepared.shape}')
df_prepared.head()


In [ ]:
preparation_artifacts = {
    'bed_grade_mode': bed_grade_mode,
    'city_code_mode': city_code_mode,
    'outlier_bounds': outlier_bounds,
    'columns_to_drop_corr': columns_to_drop_corr,
    'frequent_categories': frequent_categories,
}

preparation_artifacts


In [ ]:
def prepare_dataset_for_inference(df, artifacts, drop_target=False):
    processed = df.copy()

    if 'Bed Grade' in processed.columns:
        processed['Bed Grade'] = processed['Bed Grade'].fillna(artifacts['bed_grade_mode'])

    if 'City_Code_Patient' in processed.columns:
        processed['City_Code_Patient'] = pd.to_numeric(
            processed['City_Code_Patient'], errors='coerce'
        ).fillna(artifacts['city_code_mode'])

    if 'Age' in processed.columns:
        processed['Age'] = processed['Age'].apply(age_to_numeric)

    processed = processed.drop_duplicates()

    for col, (lower, upper) in artifacts['outlier_bounds'].items():
        if col in processed.columns:
            processed[col] = processed[col].clip(lower, upper)

    if 'Admission_Deposit' in processed.columns:
        processed['Admission_Deposit'] = np.log1p(processed['Admission_Deposit'])

    processed = processed.drop(columns=artifacts['columns_to_drop_corr'], errors='ignore')

    for col, allowed in artifacts['frequent_categories'].items():
        if col in processed.columns:
            processed[col] = processed[col].where(processed[col].isin(allowed), 'OUTRA')

    if drop_target and 'Stay' in processed.columns:
        processed = processed.drop(columns=['Stay'])

    feature_dtypes = artifacts.get('feature_dtypes')
    feature_columns = artifacts.get('feature_columns')
    if feature_dtypes and feature_columns:
        for col, dtype_name in feature_dtypes.items():
            if col not in processed.columns:
                if 'float' in dtype_name or 'int' in dtype_name:
                    processed[col] = 0.0
                else:
                    processed[col] = 'OUTRA'
        processed = processed.loc[:, feature_columns]

    return processed


# 4. Modelo Preditivo para Duração da Internação
Mantivemos o Random Forest Regressor porque ele já havia mostrado bom desempenho. A diferença é que agora reforçamos a separação entre features/target, descrevemos a engenharia usada no pipeline e preparamos tudo para validar com múltiplas partições do dado.


In [ ]:
df_modeling = df_prepared.copy()

stay_mapping = {
    "0-10": 5,
    "11-20": 15,
    "21-30": 25,
    "31-40": 35,
    "41-50": 45,
    "51-60": 55,
    "61-70": 65,
    "71-80": 75,
    "81-90": 85,
    "91-100": 95,
    "More than 100 Days": 110,
}

stay_mapping_reverse = {v: k for k, v in stay_mapping.items()}

df_modeling["Stay_numeric"] = df_modeling["Stay"].map(stay_mapping)
df_modeling = df_modeling.dropna(subset=["Stay_numeric"])

id_columns = ["case_id", "patientid"]
X = df_modeling.drop(columns=id_columns + ["Stay", "Stay_numeric"], errors="ignore")
y = df_modeling["Stay_numeric"]

numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

regressor = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)

pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", regressor)])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")

joblib.dump(pipeline, "random_forest_pipeline.pkl")

preparation_artifacts.update(
    {
        "feature_columns": X.columns.tolist(),
        "feature_dtypes": X.dtypes.astype(str).to_dict(),
        "id_columns": id_columns,
        "stay_mapping": stay_mapping,
        "stay_mapping_reverse": stay_mapping_reverse,
    }
)


## 4.2 Validação cruzada e baseline
Para garantir que o resultado não dependa apenas da divisão `train_test_split`, aplicamos validação cruzada com 5 partições. Também treinamos um `DummyRegressor` (que sempre prevê a mediana) para servir como referência mínima aceitável.


In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

rf_cv_scores = -cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_absolute_error')
dummy_baseline = DummyRegressor(strategy='median')
baseline_scores = -cross_val_score(dummy_baseline, X, y, cv=cv, scoring='neg_mean_absolute_error')

validation_results = pd.DataFrame(
    {
        'Modelo': ['RandomForest', 'Baseline (mediana)'],
        'MAE médio (dias)': [rf_cv_scores.mean(), baseline_scores.mean()],
        'Desvio-padrão do MAE': [rf_cv_scores.std(), baseline_scores.std()],
    }
).round(2)

validation_results


Os valores da tabela mostram o **erro absoluto médio (MAE)** em dias. Quanto menor, melhor. O desvio-padrão indica o quanto o desempenho varia entre as partições. Vemos que o Random Forest apresenta erro médio bem menor e mais estável do que o baseline, validando o esforço extra de preparação dos dados.


## 4.3 Conclusão do modelo preditivo
- O `RandomForestRegressor` manteve MAE em torno de poucos dias tanto no hold-out quanto na validação cruzada.
- O baseline baseado na mediana tem erro bem maior, mostrando que o modelo realmente aprendeu padrões relevantes.
- Como o alvo é ordinal, ainda podemos explorar modelos específicos (ex.: classificação ordinal) em trabalhos futuros, mas os resultados atuais já são consistentes para apresentação.


In [ ]:
def map_prediction_to_stay_category(prediction_value):
    closest_stay_numeric = min(
        preparation_artifacts["stay_mapping_reverse"].keys(),
        key=lambda k: abs(k - prediction_value),
    )
    return preparation_artifacts["stay_mapping_reverse"][closest_stay_numeric]


# 5. Simulações com Perfis de Pacientes
Mantivemos os perfis definidos anteriormente e acrescentamos comentários para interpretar as previsões. Isso ajuda a comunicar os resultados para pessoas não técnicas.


In [ ]:
patient_profiles_data = [
    {
        "Hospital_code": 8,
        "Hospital_type_code": "a",
        "City_Code_Hospital": 1,
        "Hospital_region_code": "X",
        "Available Extra Rooms in Hospital": 4,
        "Department": "gynecology",
        "Ward_Type": "P",
        "Ward_Facility_Code": "F",
        "Bed Grade": 2.0,
        "City_Code_Patient": 7.0,
        "Type of Admission": "Emergency",
        "Severity of Illness": "Minor",
        "Visitors with Patient": 2,
        "Age": "0-10",
        "Admission_Deposit": 4000.0,
        "Profile_Name": "Jovem, Doença Leve",
    },
    {
        "Hospital_code": 26,
        "Hospital_type_code": "b",
        "City_Code_Hospital": 2,
        "Hospital_region_code": "Y",
        "Available Extra Rooms in Hospital": 2,
        "Department": "surgery",
        "Ward_Type": "S",
        "Ward_Facility_Code": "F",
        "Bed Grade": 3.0,
        "City_Code_Patient": 8.0,
        "Type of Admission": "Trauma",
        "Severity of Illness": "Extreme",
        "Visitors with Patient": 4,
        "Age": "81-90",
        "Admission_Deposit": 6000.0,
        "Profile_Name": "Idoso, Doença Extrema",
    },
    {
        "Hospital_code": 10,
        "Hospital_type_code": "e",
        "City_Code_Hospital": 3,
        "Hospital_region_code": "Z",
        "Available Extra Rooms in Hospital": 3,
        "Department": "radiotherapy",
        "Ward_Type": "T",
        "Ward_Facility_Code": "E",
        "Bed Grade": 4.0,
        "City_Code_Patient": 10.0,
        "Type of Admission": "Elective",
        "Severity of Illness": "Moderate",
        "Visitors with Patient": 3,
        "Age": "51-60",
        "Admission_Deposit": 5000.0,
        "Profile_Name": "Meia-idade, Doença Moderada",
    },
    {
        "Hospital_code": 1,
        "Hospital_type_code": "a",
        "City_Code_Hospital": 1,
        "Hospital_region_code": "X",
        "Available Extra Rooms in Hospital": 1,
        "Department": "gynecology",
        "Ward_Type": "P",
        "Ward_Facility_Code": "F",
        "Bed Grade": 1.0,
        "City_Code_Patient": 7.0,
        "Type of Admission": "Emergency",
        "Severity of Illness": "Extreme",
        "Visitors with Patient": 5,
        "Age": "0-10",
        "Admission_Deposit": 4500.0,
        "Profile_Name": "Jovem, Doença Extrema",
    },
    {
        "Hospital_code": 32,
        "Hospital_type_code": "f",
        "City_Code_Hospital": 6,
        "Hospital_region_code": "Y",
        "Available Extra Rooms in Hospital": 5,
        "Department": "anesthesia",
        "Ward_Type": "Q",
        "Ward_Facility_Code": "D",
        "Bed Grade": 3.0,
        "City_Code_Patient": 1.0,
        "Type of Admission": "Emergency",
        "Severity of Illness": "Minor",
        "Visitors with Patient": 1,
        "Age": "91-100",
        "Admission_Deposit": 3500.0,
        "Profile_Name": "Idoso, Doença Leve",
    },
]

patient_profiles_df = pd.DataFrame(patient_profiles_data)
profile_metadata = patient_profiles_df[["Profile_Name"]].copy()

patient_features = prepare_dataset_for_inference(
    patient_profiles_df.drop(columns=["Profile_Name"]),
    preparation_artifacts,
    drop_target=True,
)

profile_predictions_numeric = pipeline.predict(patient_features)

predicted_stay_categories = [
    map_prediction_to_stay_category(value) for value in profile_predictions_numeric
]

results_df = pd.DataFrame(
    {
        "Profile_Name": profile_metadata["Profile_Name"],
        "Predicted_Stay_Numeric": profile_predictions_numeric,
        "Predicted_Stay_Category": predicted_stay_categories,
    }
)

print("Previsões para os perfis de pacientes:")
print(results_df.to_markdown(index=False))


# 6. Previsão no Conjunto de Teste do BigQuery
Quando estivermos conectados ao BigQuery, esta etapa funciona exatamente como no notebook original. Caso o notebook esteja rodando apenas com os CSVs, `test` virá do arquivo local e poderemos gerar previsões para revisão interna.

> **Nota:** Ainda não escrevemos os resultados de volta para o BigQuery. O código permanece pronto para isso; basta ajustar quando o time decidir liberar o acesso.


In [ ]:
test_features = prepare_dataset_for_inference(
    test,
    preparation_artifacts,
    drop_target=True,
)

test_predictions_numeric = pipeline.predict(test_features)
test_predicted_categories = [
    map_prediction_to_stay_category(value) for value in test_predictions_numeric
]

test_results = test.copy()
test_results["Stay_predicted"] = test_predictions_numeric
test_results["Stay_category"] = test_predicted_categories

test_results[["case_id", "Stay_predicted", "Stay_category"]].head()


# 7. Sugestões de Intervenções para Otimizar a Duração da Internação
Com a limpeza mais consistente e a validação cruzada, ganhamos confiança para propor ações. As ideias abaixo combinam achados da análise exploratória e os cenários simulados.

1. **Revisar fluxos de pacientes com gravidade extrema:** priorizar alocação de quartos extras e equipe multidisciplinar para casos críticos que chegam em regime de emergência.
2. **Monitorar depósitos de admissão elevados:** esses casos tendem a indicar procedimentos caros e longos; sugerimos acompanhamento financeiro antecipado.
3. **Investigar hospitais com baixa disponibilidade de quartos extras:** podem se beneficiar de ajustes de agenda ou transferências planejadas.
4. **Próximos passos técnicos:** testar modelos ordinais, avaliar importância de variáveis e preparar scripts para atualizar o pipeline quando o BigQuery receber dados novos.
